## 과제 6: 프랙털 시각화

In [67]:
# 필요 라이브러리
import numpy as np  
import matplotlib.pyplot as plt  
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [68]:
# 시각화 설정 코드
%matplotlib qt5
%config InlineBackend.figure_format = 'svg'

plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['font.size'] = 13
plt.ion()

In [69]:
# 간단한 복소수 함수 살펴보기
# 위의 함수를 구현하기 위해 z를 입력받아 제곱을 구한 뒤 1을 뺄샘하는 Python 함수 f(z)를 구현하세요.
# [+] 함수 구현
def f(z):
    return np.power(z, 2) - 1

In [70]:
# 함수 테스트
f(0)

-1

In [71]:
# 다음 세 개의 복소수(4, 1-0.2j, 1.6)를 입력으로 해서 함수를 호출하세요.
z = [4, 1-0.2j, 1.6]
f(z)

array([15.  +0.j , -0.04-0.4j,  1.56+0.j ])

In [72]:
# [+] np.linspace()를 이용하여 x축, y축 값 집합을 생성
x, y = np.meshgrid(np.linspace(-10, 10, 20), np.linspace(-10, 10, 20))

In [73]:
# [+] 복소수 집합(mesh)으로 변환하기
mesh = y * 1j + x

In [74]:
# [+] 함수 계산 결과의 절대값 구하기
output = np.abs(f(mesh))

In [75]:
# 시각화
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.scatter(x, y, output, alpha=0.2)

ax.set_xlabel('Real axis')      # 실수부
ax.set_ylabel('Imaginary axis') # 허수부
ax.set_zlabel('Absolute value') # 함수값의 절대값
ax.set_title('One Iteration: $f(z) = z^2 - 1$');

In [76]:
output = np.abs(f(f(mesh)))

ax = plt.axes(projection='3d')

ax.scatter(x, y, output, alpha=0.2)

ax.set_xlabel('Real axis')
ax.set_ylabel('Imaginary axis')
ax.set_zlabel('Absolute value')
ax.set_title('Two Iterations: $ f(z) = z^2 - 1$');

In [77]:
# 각 값에 대해 함수를 10회 반복 적용하면 어떤 결과가 나타나는지 살펴보겠습니다.
selected_values = np.array([0.4 + 0.4j, 0.41 + 0.4j, 0.4 + 0.41j])
num_iter = 9

outputs = np.zeros(
    (num_iter+1, selected_values.shape[0]),  # 10X3 Zero 배열 생성
    dtype=complex  # 데이터 타입: 복소수
)
outputs[0] = selected_values   # 함수 초기 조건

for i in range(num_iter):
    outputs[i+1] = f(outputs[i])  # 10번 함수 반복하는 동안 각 결과를 저장

fig, axes = plt.subplots(1, selected_values.shape[0], figsize=(16, 6))
axes[1].set_xlabel('Real axis')
axes[0].set_ylabel('Imaginary axis')

for ax, data in zip(axes, outputs.T):
    cycle = ax.scatter(
        data.real,  # 실수부
        data.imag,  # 허수부
        c=range(data.shape[0]), 
        alpha=0.6
    )
    ax.set_title(f'Mapping of iterations on {data[0]}')

fig.colorbar(cycle, ax=axes, location='bottom', label='Iteration');

In [78]:
# divergence_rate는 mesh의 각 복소수 값에 대해 함수 수행을 반복하고 값이 발산하기 전에 반복 횟수를 집계하는 코드입니다.
# 절차를 수행하는 코드를 완성해서 값 별 발산 전 반복횟수를 구하세요.
def divergence_rate(mesh, num_iter=10, radius=2):

    z = mesh.copy()
    # 값 별 반복 횟수를 저장
    diverge_len = np.zeros(mesh.shape)

    # 발산되지 않은 값들에 대해 함수를 반복 적용
    for i in range(num_iter):
        # [+] 1. Boolean 배열  마스크 생성
        conv_mask = np.abs(z) < radius

        # [+] 2. 해당 값들에 대한 diverge_len 값 증가
        diverge_len[conv_mask] += 1
                    
        # [+] 3. 해당 값들에 대해 함수 적용
        z[conv_mask] = f(z[conv_mask])
        
    return diverge_len

In [79]:
# divergence_rate함수를 호출하고 반환받은 diverge_len을 2D 이미지로 시각화
x, y = np.meshgrid(np.linspace(-2, 2, 400), np.linspace(-2, 2, 400))
mesh = x + (1j * y)

output = divergence_rate(mesh)

fig = plt.figure(figsize=(5, 5))
ax = plt.axes()

ax.set_title('$f(z) = z^2-1$')
ax.set_xlabel('Real axis')
ax.set_ylabel('Imaginary axis')

im = ax.imshow(output, extent=[-2, 2, -2, 2])
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.1)
plt.colorbar(im, cax=cax, label='Number of iterations');